In [ ]:
# setup for testing
import foxesscloud as f
import private as my
import json

f.username = my.fox_username
f.password = my.fox_password
f.device_sn = my.fox_device_sn
f.time_zone = "Europe/London"

f.debug_setting = 1

In [ ]:
f.get_info()

In [ ]:
f.messages['en']['0']

In [ ]:
f.get_status()

In [ ]:
f.get_status(station=1)

In [ ]:
f.get_site()

In [ ]:
f.get_logger()

In [ ]:
result = f.get_device()
print(json.dumps(result, indent=2))
print(f"generationToday = {result['generationToday']} kWh")
print(f"generationTotal = {result['generationTotal']} kWh")


In [ ]:
f.get_firmware()

In [ ]:
#f.get_work_mode()
#f.set_work_mode('Feedin')
f.get_work_mode()

In [ ]:
f.get_battery()

In [ ]:
f.get_settings()

In [ ]:
f.get_earnings()

In [ ]:
f.raw_vars[:10]

In [ ]:
f.get_raw(v='generationPower', summary=1)

In [ ]:
f.get_raw(v='gridConsumptionPower', summary=-1)

In [ ]:
f.get_raw('week', d='2023-10-21',v=['RVolt'], summary=2, station=1)

In [ ]:
f.get_raw('day', d='2023-08-21',v=['pvPower'], summary=2)

In [ ]:
f.get_raw('week', d='2023-08-21',v=['pvPower'], summary=2)

In [ ]:
f.get_raw('day', d='2023-09-01',v=['RPower'], summary=2, save='rpower')

In [ ]:
f.get_raw('day', d='2023-09-01',v=['RPower'], summary=2, load='rpower_raw_day_20230901.txt')

In [ ]:
f.get_raw('day', d='2023-08-21',v=['pvPower','meterPower2'], summary=2)

In [ ]:
d = '2023-11-17 00:00:00'
result=f.get_raw('day', d=d, v=f.power_vars, summary=3)
if result is not None:
    for r in result:
        print(f"{r['name']:20} = {round(r['kwh'],3)}")

In [ ]:
result=f.get_report('month', d=d, v=f.report_vars)
i = int(d[8:10])-1
if result is not None:
    for r in result:
        print(f"{r['variable']:20} = {round(r['data'][i]['value'],3)}")

In [ ]:
f.get_report('day', d=d, v='feedin', summary=2)

In [ ]:
f.get_report('week', d=d, v='feedin', summary=2, save='feedin')

In [ ]:
f.get_report('week', v='loads')

In [ ]:
f.get_report('year', d=d, v='generation')

In [ ]:
for d in f.date_list('2023-01-01', '2023-02-01'):
    result = f.get_raw('day', d, v=['pvPower'], summary=2)
    if result is not None:
        max = result[0].get('max')
        if max is not None and max > 0 and max < 100:
            print(f"{d} is valid, max pvPower = {max:.3f}kW")
            break

In [ ]:
for d in f.date_list('2023-01-17', '2023-01-30'):
    result = f.get_raw('day', d, v=['pv1Volt', 'pv1Current', 'pv2Volt', 'pv2Current', 'pvPower'], summary=1)
    if result is not None:
        wh = 0.0
        for i,data in enumerate(result[0]['data']):
            wh += result[0]['data'][i]['value'] * result[1]['data'][i]['value'] / 12   # pv1Volt * pv1Current / 12 
            wh += result[2]['data'][i]['value'] * result[3]['data'][i]['value'] / 12   # pv2Volt * pv2Current / 12
        print(f"{d}: generation from IV = {round(wh/1000,3):.3f}kWh, generation from pvPower = {result[4]['kwh']:.3f}kWh")

In [ ]:
result=f.get_raw('day', d=f.date_list(span='2days', today=1), v=f.power_vars, plot=1)

In [ ]:
result=f.get_raw('day', d=f.date_list(span='2days', today=1), v=['pv1Volt', 'pv2Volt', 'pv1Current', 'pv2Current'], plot=2)

In [ ]:
result=f.get_report('day', d=f.date_list(span='2days'), v=f.report_vars, summary=1, plot=1, station=1)
#result

In [ ]:
result=f.get_report('day', d=f.date_list(span='2days'), v=f.report_vars, summary=1, plot=1)

In [ ]:
result=f.get_report('day', d=f.date_list(span='day', e='2023-03-26'), v=f.report_vars, summary=1, plot=2)

In [ ]:
result=f.get_report('week', d=f.date_list(), v=f.report_vars, summary=1, plot=2)

In [ ]:
result=f.get_report('month', d=f.date_list(), v=['generation', 'feedin', 'gridConsumption'], summary=1, plot=2)

In [ ]:
result=f.get_report('year', d=f.date_list(), v=f.report_vars, summary=1, plot=2)
#print(result)

In [ ]:
# test conversion of report data to total and 24 hour profile
result = f.report_value_profile(f.get_report('day', f.date_list(span='week',today=1)[-3:], v='loads'))
print(result)

In [ ]:
# test fix_value for day report
f.fix_values = 0
result = f.get_report('day', d='2023-09-18', v='loads')
if result is not None:
    print(result[0]['total'])
f.fix_values = 1
f.get_report('day', d='2023-09-18', v='loads')

In [ ]:
# test fix_value for month report
f.get_report('month', d='2023-09-18', v='loads')

In [ ]:
# test fix_value for year report
f.get_report('year', d='2023-09-18', v='loads')

In [ ]:
f.get_schedule()

In [ ]:
period = [f.set_period(7,12,'SelfUse'), f.set_period(16,19,'Feedin'), f.set_period(2,4,'ForceCharge')]
f.set_schedule(periods=period)
f.set_schedule(enable=0)

In [ ]:
f.british_summer_time()

In [ ]:
# test british summer time
import datetime
dates = f.date_list(span='year', today=2, s=datetime.datetime.now())
flags = f.british_summer_time(dates)
last = None
for date, flag in zip(dates, flags):
    if last is not None and last != flag:
        print(f"{date} = {flag}")
    last = flag

In [ ]:
import datetime
dat = datetime.datetime.strptime('2024-03-31 00:00', '%Y-%m-%d %H:%M')
for i in range(0,4):
    print(f"{dat} = {f.british_summer_time(dat)}")
    dat = dat + datetime.timedelta(hours=1)

In [ ]:
f.get_report('day', d=f.date_list(span='weekday',e='2023-10-10',today=2),v='loads',summary=2)

In [ ]:
template = f.find_template('Daily')
print(f"found: {template}")
f.templates

In [ ]:
f.debug_setting=1
f.set_schedule(template='DailyRunning')
#f.set_schedule(template=template)

In [ ]:
f.set_schedule(enable=0)

In [ ]:
f.set_work_mode('Feedin')

In [ ]:
print(f.get_cell_temps())

In [ ]:
print(f.get_cell_volts())

In [ ]:
f.set_charge(st1=2,en1=5,ch1=1)